# Crop YOLOv4 people

## Path definitions - change to your own

In [4]:
IMAGES_PATH = "/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedImgs/"
YOLOV4_RESULTS_CSV = "/content/drive/MyDrive/TFM/yolov4_results/transform.csv"
OUT_IMAGES = "/content/drive/MyDrive/TFM/yolov4_crop"

Here Google Drive storage is loaded. You can omit this cell if you are not using it.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%mkdir {OUT_IMAGES}

In [6]:
import pandas as pd

In [11]:
df = pd.read_csv(YOLOV4_RESULTS_CSV)
df

,Unnamed: 0,filename,x,y,x2,y2,class
0,0,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,112.0,50.0,332.0,925.0,person
1,1,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,745.0,248.0,854.0,626.0,person
2,2,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,752.0,240.0,893.0,628.0,person
3,3,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1549.0,159.0,1828.0,911.0,person
4,4,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1498.0,181.0,1652.0,795.0,person
...,...,...,...,...,...,...,...
2525,2525,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1215.0,328.0,1427.0,884.0,person
2526,2526,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,431.0,365.0,500.0,557.0,person
2527,2527,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1316.0,346.0,1641.0,1077.0,person
2528,2528,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1303.0,337.0,1595.0,1039.0,person
